In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

### LOGIN

In [2]:
driver = webdriver.Chrome()
driver.get("https://pabx.flux.net.br/2cd33c87b7424360833b8281b64046ea/index.php")

# Localiza os campos de login e senha
username_field = driver.find_element(By.NAME, "login") 
password_field = driver.find_element(By.NAME, "pass")  

# Insere as credenciais
username_field.send_keys("FazComexRelatorios")  
password_field.send_keys("relatorios@2023")   

# Pressiona o botão de login
password_field.send_keys(Keys.RETURN)

time.sleep(5)

### Filtro de Data

In [3]:
# Configuração de escrita data D-1
start_datetime = (datetime.now() - timedelta(days=1)).strftime("%d/%m/%Y 00:00:00")
end_datetime = (datetime.now() - timedelta(days=1)).strftime("%d/%m/%Y 23:59:59")    

# Localiza os campos de filtro de data
start_date_field = driver.find_element(By.ID, "data_inicial")  
end_date_field = driver.find_element(By.ID, "data_final")      

# Preenche os campos com a data de ontem conforme configuração
start_date_field.clear()
start_date_field.send_keys(start_datetime)  
end_date_field.clear()
end_date_field.send_keys(end_datetime)      

# Clica no botão para aplicar o filtro
search_button = driver.find_element(By.CLASS_NAME, "btn-primary") 
search_button.click()

### Download Tabela

In [4]:
download_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.ID, "botao_csv"))
)

download_button.click()
time.sleep(4)
driver.quit()

### Atualização incremental

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Definir o diretório onde o arquivo CSV é baixado
diretorio_downloads = r"C:\Users\Bruno\Downloads"

# Procurar pelo arquivo que começa com "relatorio_ligacoes" no diretório
arquivos_csv = glob.glob(os.path.join(diretorio_downloads, "relatorio_ligacoes_*.csv"))

if arquivos_csv:
    # Selecionar o arquivo mais recente
    arquivo_csv = max(arquivos_csv, key=os.path.getctime)
    print(f"Arquivo detectado: {arquivo_csv}")
else:
    print("Nenhum arquivo encontrado com o padrão especificado.")
    exit()  # Encerra o script se nenhum arquivo for encontrado

Arquivo detectado: C:\Users\Bruno\Downloads\relatorio_ligacoes_09_05_2025.csv


In [ ]:
caminho_arquivo_xlsx = r"C:\Users\Bruno\Documents\ligacoes\ligacoes_2024_25.xlsx"

# Carregar a tabela existente de ligações em XLSX
tabela_ligacoes = pd.read_excel(caminho_arquivo_xlsx)

# Carregar o arquivo CSV detectado
novos_dados = pd.read_csv(arquivo_csv)

# Fazer um merge incremental com base na coluna ID
df = pd.concat([tabela_ligacoes, novos_dados]).drop_duplicates(subset=['ID'], keep='last')
df = df.iloc[:-1]

# Salva o excel atualizado na pasta
df.to_excel(caminho_arquivo_xlsx, index=False)
print(f"Arquivo Excel atualizado salvo em {caminho_arquivo_xlsx}")


Arquivo Excel atualizado salvo em C:\Users\Bruno\Documents\ligacoes\ligacoes_2024_25.xlsx


### Transformações

In [ ]:
# conversão de colunas
df['ID'] = df['ID'].astype(str)
df["HORA"] = pd.to_datetime(df["HORA"], format='%d/%m/%Y %H:%M:%S')
df['Nº A'] = df['Nº A'].astype(str)
df['Nº B'] = df['Nº B'].astype(str)
df['RAMAL'] = df['RAMAL'].astype(str)
df['SENTIDO'] = df['SENTIDO'].astype(str)
df['DURAÇÃO'] = pd.to_timedelta(df['DURAÇÃO']).dt.total_seconds().astype(int)

df.dtypes

ID                  object
HORA        datetime64[ns]
EVENTO              object
Nº A                object
Nº B                object
RAMAL               object
SENTIDO             object
DURAÇÃO              int64
FILA                object
GRAVAÇÃO            object
dtype: object

In [ ]:
df_new = df.drop(columns=['FILA','GRAVAÇÃO'])

df_new.columns = df_new.columns.str.lower()

df_new = df_new.rename(columns={'nº a':'origem_lig','nº b':'dest_lig','hora':'data_lig','duração':'tempo_chamada'})

df_new['dest_lig'] = df_new['dest_lig'].str.replace(r'^\+?55', '', regex=True)
df_new['ramal'] = df_new['ramal'].apply(lambda x: x[:-2])

# Remover '.0' das colunas 'origem_lig' e 'dest_lig' (quando existir)
df_new['origem_lig'] = df_new['origem_lig'].astype(str).str.replace(r'\.0$', '', regex=True)
df_new['dest_lig'] = df_new['dest_lig'].astype(str).str.replace(r'\.0$', '', regex=True)

mapa_ramal = {
    '400': 'Financeiro',
    '401': 'Inácio Moraes',
    '402': 'Thayane Soares', #Andressa Kunt (trocar para andressa desde a entrada dela sem modificar as da thayane)
    '403': 'Luis Correa',
    '404': 'Larissa Melo',
    '405': 'Greice Nunes',
    '406': 'Leonardo Maciel'
}

mapa_email_aten = {
    '400': 'financeiro@fazcomex.com.br',
    '401': 'inacio.moraes@fazcomex.com.br',
    '402': 'thayane.soares@fazcomex.com.br', #andressa.kunt@fazcomex.com.br
    '403': 'luis.correa@fazcomex.com.br',
    '404': 'larissa.melo@fazcomex.com.br',
    '405': 'greice.nunes@fazcomex.com.br',
    '406': 'leonardo.maciel@fazcomex.com.br'
}

# nova coluna 'ramal_owner' baseada no valor de 'ramal'
df_new['ramal_owner'] = df_new['ramal'].map(mapa_ramal).fillna('Outro')

# nova coluna de email pra aplicar RLS na base
df_new['email_aten_rls'] = df_new['ramal'].map(mapa_email_aten).fillna('não mapeado')

# nova coluna de email administrativo pro adm ver todos os dados
df_new['email_admin_rls'] = 'leandro.sprenger@fazcomex.com.br'
df_new.head(3)

,id,data_lig,evento,origem_lig,dest_lig,ramal,sentido,tempo_chamada,ramal_owner,email_aten_rls,email_admin_rls
0,1704207224.494384,2024-01-02 11:53:44,Atendida,1142999147,5131910990,n,E,0,Outro,não mapeado,leandro.sprenger@fazcomex.com.br
1,1704208976.494873,2024-01-02 12:22:56,Atendida,19999155812,5125000770,n,E,0,Outro,não mapeado,leandro.sprenger@fazcomex.com.br
2,1704209001.494877,2024-01-02 12:23:21,Atendida,19999155812,5125000770,n,E,0,Outro,não mapeado,leandro.sprenger@fazcomex.com.br


In [ ]:
df_new.to_csv("tb_ligacoes.csv", index=False)